<a href="https://colab.research.google.com/github/beatLaboratory/TIMBRE/blob/main/LFP_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run TIMBRE on real data

In this notebook we import the experimental data and use LFP and spikes to predict which maze arm the rat is occupying.

First, let's install TIMBRE and its dependencies.

In [ ]:
#!git clone https://github.com/beatLaboratory/TIMBRE.git
#!pip install -r requirements.txt
#!pip install requests
!pip install scipy

# Download data

We will download LFP, spike, and behavioral data. We will use experimental session 4 because it is smaller and therefore faster to run.

In [ ]:
import requests
import os
from scipy import io as iomat
import io

import numpy as np
import rat.helpers as helpers

repository_id = "24757638"  # Behavior_and_spiking_data_for_rats_running_a_3-arm_maze
url = f"https://api.figshare.com/v2/articles/{repository_id}"

# Make the API request
response = requests.get(url)
files = response.json()['files']

file_pattern = "data04.mat"

# Find the matching files
file = next((file for file in files if file['name'] == file_pattern), None)

if file:
    # Download the file using requests (previously wget, but not universal across shells)
    download_url = file['download_url']
    file_name = file['name']
    print(f"Downloading file: {file_name}")
    response = requests.get(download_url)
    
    # Load the .mat file directly from the response content
    data = iomat.loadmat(io.BytesIO(response.content))

LFPs = helpers.filter_data(data['lfps'], 2, fs=25, use_hilbert=True)


# Visualize network behavior

We will train TIMBRE with different hidden layer sizes and look at the average response in each layer as a function of position along the track. We do this for the phase in which the animal is running towards the reward port while in one of the 3 maze arms. Note that for larger hidden layer sizes, the hidden nodes' responses are localized in position, even though this information is not provided to the network during training.

In [ ]:
from matplotlib import pyplot as plt
from TIMBRE import TIMBRE

fig, axs = plt.subplots(4, 4, figsize=(20, 5))

trial_phase = 1
n_folds = 5
which_fold = 0

lapID = data['lapID']

test_inds, train_inds = helpers.test_train(data['lapID'], trial_phase, n_folds, which_fold)  # only test 1 fold per session
wLFPs, _, _ = helpers.whiten(LFPs, train_inds)

# Verify the shapes of the indices arrays
print(f"Test indices shape: {test_inds.shape}")
print(f"Train indices shape: {train_inds.shape}")

# Additional debug statements to verify input to TIMBRE
print(f"X (wLFPs) shape: {wLFPs.shape}")
print(f"Y (lapID[:, 1]) shape: {lapID[:, 1].shape}")
print(f"inds_train shape: {train_inds.shape}")
print(f"inds_test shape: {test_inds.shape}")
print(f"X[inds_train, :] shape: {wLFPs[train_inds, :].shape}")
print(f"Y[inds_train] shape: {lapID[train_inds, 1].shape}")
print(f"X[inds_test, :] shape: {wLFPs[test_inds, :].shape}")
print(f"Y[inds_test] shape: {lapID[test_inds, 1].shape}")

n_bins = 20
pos_binned = helpers.group_by_pos(data['lapID'][:, 4], n_bins, train_inds)  # Convert position along the track into discrete bins.
arm_and_pos_binned = data['lapID'][:, 1] * n_bins + pos_binned  # Represent arm x position as integer between 0-19 (arm 1), 20-39 (arm 2), 40-59 (arm 3)



titles = ['Projection (real part)', 'Amplitude', 'Softmax 1', 'Softmax 2 (Output)'];
for i in range(axs.shape[0]):
    print(f"Training network {i + 1} of {axs.shape[0]} (hidden layer size {3 * 2 ** i})")  # try 4 different hidden layer sizes
    m, _, _ = TIMBRE(wLFPs, data['lapID'][:, 1], test_inds, train_inds, hidden_nodes=3 * 2 ** i)  # train network
    for j in range(axs.shape[1]):  # Loop through each layer
        p = helpers.layer_output(wLFPs[test_inds], m, j)  # Calculate layer's response to input, using only test data
        if j == 0:
            p = p[:, :p.shape[1] // 2]  # just get real component for complex-valued output
            axs[i, 0].set_ylabel(str(3 * 2 ** i) + ' features');
        if i == 0:
            axs[0, j].set_title((titles[j]));
        axs[i, j].plot(helpers.accumarray(arm_and_pos_binned[test_inds], p));  # plot mean response of layer to test data as a function of position
        axs[i, j].autoscale(enable=True, axis='both', tight=True);
        if i < axs.shape[0] - 1:
            axs[i, j].set_xticks([]);
        else:
            axs[i, j].set_xlabel('Position');


# Compare TIMBRE to other models

Now we will compare the performance of three classifiers:
1.   **Carrier-based** - Linear classifier trained on the demodulated LFP. During running, the track is further subdivided into 8 bins so that the classifier can accommodate changes in place code along the maze arm.
2.   **Carrier-free (TIMBRE)** - a complex-valued neural network for identifying phase-amplitude patterns in the LFP. We do not need to feed in the position along the track (as done with carrier-based decoding) since the hidden layer learns to tile each track without additional labels (see previous section).
3.   **Spike-based** - Linear classifier trained on the firing rates of the neurons that were simultaneously recorded.

In [ ]:
from sklearn.linear_model import LogisticRegression
from TIMBRE import carrier_based

n_folds = 5  # how many folds to split data into
run_folds = 1  # how many folds to train (can be up to n_folds; using just 1 for speed)
all_scores = np.zeros((3, 2, run_folds))

for i in range(run_folds):
    for j in range(all_scores.shape[1]):
        test_inds, train_inds = helpers.test_train(data['lapID'], j + 1, n_folds, i)  # only test 1 fold per session
        wLFPs, _, _ = helpers.whiten(LFPs, train_inds)
        if j == 0:  # during running, subdivide track for carrier-based decoding
            gp = helpers.group_by_pos(data['lapID'][:, 4], 8, train_inds)
        else:
            gp = []
        _, _, all_scores[0, j, i] = carrier_based(wLFPs, data['lapID'][:, 1], test_inds, train_inds, subgroups=gp)
        _, _, all_scores[1, j, i] = TIMBRE(wLFPs, data['lapID'][:, 1], test_inds, train_inds, 3 * 2 ** 3)  # 8 hidden nodes per arm
        model_sp = LogisticRegression(max_iter=300)
        model_sp.fit(data['spikes'][train_inds], data['lapID'][train_inds, 1])
        all_scores[2, j, i] = np.mean(data['lapID'][test_inds, 1] == model_sp.predict(data['spikes'][test_inds]))

# performance of 3 models (carrier-based, carrier-free, and spike-based) across 2 conditions (running, staying)
print(np.mean(all_scores, axis=2))
